In [ ]:
import os
import glob

import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf

%matplotlib inline

In [ ]:
# データ読み込み
data = np.load(open("mnist_mini_comp.npz", "br"))

train_image = data["train_image"].reshape(-1, 28, 28, 1)
train_label = data["train_label"]
valid_image = data["valid_image"].reshape(-1, 28, 28, 1)
valid_label = data["valid_label"]
test_image = data["test_image"].reshape(-1, 28, 28, 1)
test_label = data["test_label"]

# settings
batch_size = 100
train_total_itr = int(train_image.shape[0] / batch_size)
eval_total_itr = int(test_image.shape[0] / batch_size)

In [ ]:
# 読み込んだデータの概要
print("train_image.shape: {}".format(train_image.shape))
print("train_label.shape: {}".format(train_label.shape))
print("")
print("valid_image.shape: {}".format(valid_image.shape))
print("valid_label.shape: {}".format(valid_label.shape))
print("")
print("test_image.shape: {}".format(test_image.shape))
print("test_label.shape: {}".format(test_label.shape))
print("")
print("dtype")
print("  train_image.dtype: {}".format(train_image.dtype))
print("  train_label.dtype: {}".format(train_label.dtype))

In [ ]:
# label側の内容確認
train_label[:10]

In [ ]:
# image側の内容確認
plt.imshow(train_image[0][:, :, 0], cmap="gray")

In [ ]:
# == placeholder定義 ==
# データ
x = tf.placeholder(dtype=tf.float32)
# 正解ラベル
t = tf.placeholder(dtype=tf.int32)

In [ ]:
# == 第1層 convolution + max pooling ==
# variable定義
W1 = tf.Variable(tf.random_normal([5, 5, 1, 6], stddev=0.1), dtype=tf.float32)
b1 = tf.Variable(tf.random_normal([6], stddev=0.1), dtype=tf.float32)

# 計算グラフ定義
pre_conv1 = tf.nn.conv2d(x, W1, (1, 1, 1, 1), "VALID")
conv1 = tf.nn.bias_add(pre_conv1, b1)
relu1 = tf.nn.relu(conv1)
h1 = tf.nn.max_pool(relu1, (1, 2, 2, 1), (1, 2, 2, 1), "VALID")

In [ ]:
# == 第2層 convolution + max pooling ==
# variable定義
# (ここにコードを入力)

# 計算グラフ定義
# (ここにコードを入力)

In [ ]:
# == 第3層 fully-connected ==
# variable定義
W3 = tf.Variable(tf.random_normal([256, 60], stddev=0.1), dtype=tf.float32)
b3 = tf.Variable(tf.random_normal([60], stddev=0.1), dtype=tf.float32)

# 計算グラフ定義
a3 = tf.matmul(h2_reshaped, W3) + b3
h3 = tf.nn.relu(a3)

In [ ]:
# == 第4層 fully-connected ==
# variable定義
# (ここにコードを入力)

# 計算グラフ定義
# ※ 出力層のactivationは logitsという変数名にしておく
# (ここにコードを入力)


In [ ]:
# == コスト定義 ==
# tf.nn.sparse_softmax_cross_entropy_with_logitsで得られるのはレコード毎のコスト
cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, labels=t)
# レコード毎の値を 平均値に集約
cost = tf.reduce_mean(cross_entropy)

In [ ]:
# == 学習用のopを定義 ==
trainer = tf.train.GradientDescentOptimizer(learning_rate=0.03)
train_op = trainer.minimize(cost, var_list=[W1, b1, W2, b2, W3, b3, W4, b4])

In [ ]:
# モデルのsave/restore用 utility
if not os.path.isdir("models"):
    os.makedirs("models")
saver = tf.train.Saver(max_to_keep=None)

In [ ]:
# 評価用の関数
def evaluation(sess, d_image, d_label):
    match_count = 0
    for i in range(eval_total_itr):
        batch_image = d_image[i * batch_size: (i + 1) * batch_size]
        batch_label = d_label[i * batch_size: (i + 1) * batch_size]
        raw_results = sess.run(logits, feed_dict={x: batch_image})
        results = np.argmax(raw_results, axis=1)
        match_count += np.sum(batch_label == results)
    accuracy = float(match_count) / float(d_label.size)
    return accuracy

In [ ]:
# 実際の学習
total_epochs = 10

cost_history = list()
valid_history = list()
test_accuracy = 0.

# 以前の実行の途中経過が残っていれば削除
fpath_pattern = os.path.join("models", "model.*")
model_list = glob.glob(fpath_pattern)
for p in model_list:
    os.remove(p)
    
with tf.Session() as sess:
    # variable初期化
    sess.run(tf.global_variables_initializer())
    
    itr = 1
    for epoch in range(1, total_epochs + 1):
        print("")
        print("=== epoch {} ===".format(epoch))
        
        for i in range(train_total_itr):
            # image, lagelについての mini-batchデータを用意
            image = train_image[i * batch_size: (i + 1) * batch_size]
            label = train_label[i * batch_size: (i + 1) * batch_size]
            
            # TensorFlowのsession実行、学習・コスト取得
            fd = {x: image, t: label}
            cost_val, _ = sess.run((cost, train_op), feed_dict=fd)
            
            if itr % 10 == 0:
                cost_history.append((itr, cost_val))
                print("itr: {}, cost: {:.04f}".format(itr, cost_val))
                
            # validation
            if itr % 50 == 0:
                acc = evaluation(sess, valid_image, valid_label)
                valid_history.append((itr, acc))
                print("validation accuracy: {}".format(acc))
                print("")
                fpath = os.path.join("models", "model.{}".format(itr))
                saver.save(sess, fpath)
                
            itr += 1
            
    # test
    sorted_valid_history = sorted(valid_history, key=lambda x: x[1], reverse=True)
    best_itr = sorted_valid_history[0][0]
    best_acc = sorted_valid_history[0][1]
    print("best validation performance was {} at iter {}".format(best_acc, best_itr))
    
    fpath = os.path.join("models", "model.{}".format(best_itr))
    saver.restore(sess, fpath)
    test_accuracy = evaluation(sess, test_image, test_label)
    print("test accuracy: {}".format(acc))

In [ ]:
plt.figure(figsize=(10, 5))
ax1 = plt.subplot(211)
ax2 = plt.subplot(212)

train_x = [t[0] for t in cost_history]
train_y = [t[1] for t in cost_history]
valid_x = [t[0] for t in valid_history]
valid_y = [t[1] for t in valid_history]

ax1.plot(train_x, train_y, color="green")
ax2.plot(valid_x, valid_y)
ax2.hlines(test_accuracy, xmin=train_x[0], xmax=train_x[-1], color="red", linestyles="--")

plt.title("learning curve for MNIST training")
ax1.set_title("cost")
ax2.set_title("accuracy")

ax2.legend(["validation", "test"], loc=4);